In [1]:
import numpy as np

class TransformCoordinates:
    
    def __init__(self, tray, position, points):
        self.tray = tray
        self.position = position
        self.points = np.array(points)
        self.rad = []
        self.deg = []
        self.center = []
        self.centerPosition = 0.
        self.newPoints = []
        
    #Calculate the distance between two points or the origin
    def distance(self, tup):
        if np.array(tup).size == 2:
            return np.sqrt(((tup[0])**2) + ((tup[1])**2))
        else:
            return np.sqrt(((tup[1][0] - tup[0][0])**2) + ((tup[1][1] - tup[0][1])**2))

    #Using law of cosines we calculate the angle between the fiducials and the P1/P2 frames
    def angleOfFrames(self, Fiducial, baseplate):
        f = self.distance(Fiducial)
        b = self.distance(baseplate)
        th1 = 0.
        if (baseplate[0][0] - baseplate[1][0]) > 0.:
            b1, b2 = (baseplate[0][0] - baseplate[1][0]), (baseplate[0][1] - baseplate[1][1])
        else:
            b1, b2 = (baseplate[1][0] - baseplate[0][0]), (baseplate[1][1] - baseplate[0][1])

        th1 = Fiducial[1][0]*b1 + Fiducial[1][1]*b2
        theta = np.pi/2 - np.arccos((th1)/(f*b))
        return theta

    #Using a rotation matrix transform the measurements of the points in the fiducial frame
    def transform(self):
        theta = self.angleOfFrames(self.tray, self.position)
        c, s = np.cos(theta), np.sin(theta)
        rotate = np.array(((c, -s), (s, c)))
        self.newPoints = np.dot((self.points - np.array(self.position)[0]), rotate.T)

    #Calculate the angle offset of the placement of the components
    #also get the center of the square that is made by the fiducial measurements
    def RunTransform(self):
        self.transform()
        for x in range(0, int(len(self.newPoints)/2)):
            r = np.arctan2((self.newPoints[x*2 + 1][1] - self.newPoints[x*2][1]), 
                           (self.newPoints[x*2 + 1][0] - self.newPoints[x*2][0]))
            self.rad.append(r)
            self.deg.append(np.degrees(r))

        self.center = np.sum(self.newPoints, axis=0)/4
        self.centerPosition = self.distance(self.center)